In [1]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

# How to Compute the Conjunctive Normal Form

Formulas are represented as nested tuples.  In order to convert a string into a nested tuple we use the *parser*
that is found in the module `propLogParser`.

In [2]:
import propLogParser as plp

The function `eliminateBiconditional(f)` takes a formula `f` from propositional logic and eliminates all occurrences of the operator '↔' from this formula.  This is done by using the following equivalence:
$$ (f \leftrightarrow g) \;\Leftrightarrow\; (f \rightarrow g) \wedge (g \rightarrow f) $$

In [3]:
def eliminateBiconditional(f):
    'Eliminate the logical operator "↔" from the formula f.'
    if isinstance(f, str):   # This case covers variables.
        return f
    if f[0] == '↔':
        g, h = f[1:]
        return eliminateBiconditional( ('∧', ('→', g, h), ('→', h, g)) )
    if f[0] == '⊤':
        return f
    if f[0] == '⊥':
        return f
    if f[0] == '¬':
        g  = f[1]
        ge = eliminateBiconditional(g)
        return ('¬', ge)
    else:   # This case covers '→', '∧', and '∨'.
        op, g, h = f
        ge       = eliminateBiconditional(g)
        he       = eliminateBiconditional(h)
        return (op, ge, he)

The function $\texttt{eliminateConditional}(f)$ takes a formula $f$ from propositional logic and eliminates all occurrences of the operator '→' from this formula.  This is done by using the following equivalence:
$$ (g \rightarrow h) \;\Leftrightarrow\; (\neg g \vee h) $$

In [5]:
def eliminateConditional(f):
    'Eliminate the logical operator "→" from f.'
    if isinstance(f, str): 
        return f
    if f[0] == '⊤':
        return f
    if f[0] == '⊥':
        return f
    if f[0] == '→':
        g, h = f[1:]
        ge   = eliminateConditional(g)
        he   = eliminateConditional(h)
        return ('∨', ('¬', ge), he)
    if f[0] == '¬':
        g  = f[1]
        ge = eliminateConditional(g)
        return ('¬', ge)
    else:      # This case covers '∧' and '∨'.
        op, g, h = f
        ge       = eliminateConditional(g)
        he       = eliminateConditional(h)
        return (op, ge, he)

The function $\texttt{nnf}(f)$ computes the *negation normal form* of $f$, while $\texttt{neg}(f)$ computes the *negation normal form* of $\neg f$.  The expression $\texttt{nnf}(f)$ is defined recursively as follows:
<ol>
    <li> $\texttt{nnf}(\neg \texttt{F}) = \texttt{neg}(\texttt{F})$, </li>
    <li> $\texttt{nnf}(\texttt{F}_1 \wedge \texttt{F}_2) = 
          \texttt{nnf}(\texttt{F}_1) \wedge \texttt{nnf}(\texttt{F}_2)$,</li>
    <li> $\texttt{nnf}(\texttt{F}_1 \vee \texttt{F}_2) = 
          \texttt{nnf}(\texttt{F}_1) \vee \texttt{nnf}(\texttt{F}_2)$.</li>
</ol>
The auxiliary function $\texttt{neg}$ is also defined recursively:
<ol>
    <li> $\texttt{neg}(p) = \texttt{nnf}(\neg p) = \neg p$ for all propositional variables $p$,</li>
    <li> $\texttt{neg}(\neg F) = \texttt{nnf}(\neg \neg F) = \texttt{nnf}(F)$,</li>
    <li> $$\begin{array}[t]{cl}
         & \texttt{neg}\bigl(F_1 \wedge F_2 \bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg(F_1 \wedge F_2)\bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg F_1 \vee \neg F_2\bigr) \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg F_1\bigr) \vee \texttt{nnf}\bigl(\neg F_2\bigr) \\[0.1cm]
       = & \texttt{neg}(F_1) \vee \texttt{neg}(F_2).
       \end{array}
      $$
      Therefore we have $\texttt{neg}\bigl(F_1 \wedge F_2 \bigr) = \texttt{neg}(F_1) \vee \texttt{neg}(F_2)$.</li>
     <li> $$\begin{array}[t]{cl}
         & \texttt{neg}\bigl(F_1 \vee F_2 \bigr)        \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg(F_1 \vee F_2) \bigr)  \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg F_1 \wedge \neg F_2 \bigr)  \\[0.1cm]
       = & \texttt{nnf}\bigl(\neg F_1\bigr) \wedge \texttt{nnf}\bigl(\neg F_2 \bigr)  \\[0.1cm]
       = & \texttt{neg}(F_1) \wedge \texttt{neg}(F_2). 
       \end{array}
      $$
      Therefore we have $\texttt{neg}\bigl(F_1 \vee F_2 \bigr) = \texttt{neg}(F_1) \wedge \texttt{neg}(F_2)$.</li>
</ol>

In [7]:
def nnf(f):
    'Compute the negation normal form of f.'
    if isinstance(f, str): 
        return f
    if f[0] == '⊤':
        return f
    if f[0] == '⊥':
        return f
    if f[0] == '¬':
        g = f[1]
        return neg(g)
    if f[0] == '∧':
        g, h = f[1:]
        return ('∧', nnf(g), nnf(h))
    if f[0] == '∨':
        g, h = f[1:]
        return ('∨', nnf(g), nnf(h))

In [6]:
def neg(f):
    'Compute the negation normal form of ¬f.'
    if isinstance(f, str): 
        return ('¬', f)
    if f[0] == '⊤':
        return ('⊥',)
    if f[0] == '⊥':
        return ('⊤',)
    if f[0] == '¬':
        g = f[1]
        return nnf(g)
    if f[0] == '∧':
        g, h = f[1:]
        return ('∨', neg(g), neg(h))
    if f[0] == '∨':
        g, h = f[1:]
        return ('∧', neg(g), neg(h))

The function $\texttt{cnf}(f)$ takes a formula $f$ that is in *negation normal form*, i.e. the negation operator is only applied to propositional variables and returns the *conjunctive normal form* of $f$ in *set notation*.  In order to achieve
this it uses the distributive law
$$ (f \wedge g) \vee (h \wedge k) \Leftrightarrow (f \vee h) \wedge (f \vee k) \wedge (g \vee h) \wedge (g \vee k). $$

In [9]:
def cnf(f):
    if isinstance(f, str):         # f is a variable
        return { frozenset({f}) }
    if f[0] == '⊤':
        return set()
    if f[0] == '⊥':
        return { frozenset() }
    if f[0] == '¬':
        return { frozenset({f}) }  # f is a negative literal
    if f[0] == '∧':
        g, h = f[1:]
        return cnf(g) | cnf(h)
    if f[0] == '∨':
        g, h = f[1:]
        return { k1 | k2 for k1 in cnf(g) for k2 in cnf(h) }

The function $\texttt{isTrivial}(C)$ checks whether the clause $C$ is *trivial*.

In [10]:
def isTrivial(Clause):
    return any(('¬', p) in Clause for p in Clause)

The function $\texttt{simplify}(Cs)$ takes a set of clauses and removes all trivial clauses from $Cs$.

In [11]:
def simplify(Clauses):
    return { C for C in Clauses if not isTrivial(C) }

The function $f$ takes a propositional formula $f$ and transforms $f$ into *conjunctive normal form*.

In [12]:
def normalize (f):
    n1 = eliminateBiconditional(f)
    n2 = eliminateConditional(n1)
    n3 = nnf(n2)
    n4 = cnf(n3)
    return simplify(n4)

In [14]:
def test(s):
    f = plp.LogicParser(s).parse()
    print(f'The knf of {s} is:')
    print(prettify(normalize(f)))

In [15]:
def prettify(M):
    """Turn the set of frozen sets M into a string that looks like a set of sets.
       M is assumed to be the power set of some set.
    """
    if M == set():
        return '{}'
    result = "{"
    for A in M:
        if A == frozenset(): 
            result += "{}, "
        else:
            result += str(set(A)) + ", " # A is converted from a frozen set to a set
    result = result[:-2] # remove the trailing substring ", "
    result += "}"
    return result

In [16]:
test('¬(a ∧ b) ↔ ¬a ∨ ¬b')

The knf of ¬(a ∧ b) ↔ ¬a ∨ ¬b is:
{}


In [17]:
test('(a → b) ↔ (¬a ∧ ¬b)')

The knf of (a → b) ↔ (¬a ∧ ¬b) is:
{{('¬', 'b'), ('¬', 'a')}, {('¬', 'b'), 'a'}, {('¬', 'b')}}


In [18]:
test('(p ∧ q → r) ∨ ¬r → ¬p')

The knf of (p ∧ q → r) ∨ ¬r → ¬p is:
{{('¬', 'p'), 'r'}, {('¬', 'p'), ('¬', 'r')}, {'q', ('¬', 'p')}}


In [19]:
test('⊤')

The knf of ⊤ is:
{}


In [20]:
test('⊥')

The knf of ⊥ is:
{{}}
